# 外皮情報LV1からLV2へのコンバート

## 1. 外皮簡易法＋用途別床面積
- 当該住戸の外皮の部位の面積等を用いずに簡易熱負荷計算を実施する場合には、部位の面積および長さを、「平成28年省エネルギー基準に準拠したエネルギー消費性能の評価に関する技術情報（住宅）」２．エネルギー消費性能の算定方法　2.2　算定方法　第三章　 暖冷房負荷と外皮性能　	第二節　外皮性能　9.当該住戸の外皮の部位の面積等を用いずに外皮性能を評価する方法　に示された標準住戸（以下、「簡易外皮入力モデル」）における部位の面積および長さ等より推定する。
- 屋根・天井、外壁、床の構成は、入力値である部位種別の熱貫流率をもとに、部位のU値から壁体構成を決定する方法により決定する。
- 土間床等の外周部の線熱貫流率については、簡易熱負荷計算での扱いに従う
- 開口部の日射熱取得率および外部日除けの取得日射熱補正係数は、それぞれ、季節別に、垂直面日射熱取得率および取得日射熱補正係数の入力値もしくは規定値を使用する。この場合、簡易熱負荷計算においては、開口部の入射角特性および外部日除けは考慮しない。
- その他は2.と同様に求める。

#### 《設計住戸の各部位の長さおよび面積の推定》
$$ \qquad
A_{i,d}= A_{i,s}・\frac{A_{floor,d}}{A_{floor,s}}
\qquad (1)
\\
$$
$$ \qquad
L_{i,d}= L_{i,s}・\frac{A_{floor,d}}{A_{floor,s}}
\qquad (2)
\\
$$

- 入力値：

  - 簡易外皮入力モデル住戸の延床面積$[m ^ 2]$：$A_{floor,s}$
  - 設計住戸の延床面積$[m ^ 2]$：$A_{floor,d}$
  - 簡易外皮入力モデル住戸の部位（一般部位又は開口部）𝑖の面積$[m ^ 2]$：$A_{i,s}$
  - 簡易外皮入力モデル住戸の部位（土間床等の外周部）𝑖の長さ$[m]$：$L_{i,s}$

- 出力値：

  - 設計住戸の部位（一般部位又は開口部）𝑖の面積の推定値$[m ^ 2]$：$A_{i,d}$
  - 設計住戸の部位（土間床等の外周部）𝑖の長さの推定値$[m]$：$L_{i,d}$

#### コンバート【分類：共通】
- 簡易外皮入力フラグをTrueとする

- レベル1：
  - 地域区分$[-]$：Region
  - 主たる居室の床面積$[m^2]$：MainOccupantRoomFloorArea
  - その他の居室の床面積$[m^2]$：OtherOccupantRoomFloowArea
  - 非居室の床面積$[m^2]$：NonOccupantRoomFloorArea
   
- レベル2：
  - 地域区分$[-]$：Region
  - <font color="red">簡易外皮入力フラグ$[-]$：FlagSimplifiedEnveopeInput</font>
  - 主たる居室の床面積$[m^2]$：MainOccupantRoomFloorArea
  - その他の居室の床面積$[m^2]$：OtherOccupantRoomFloowArea
  - 非居室の床面積$[m^2]$：NonOccupantRoomFloorArea

In [3]:
data1_common = {
    'Region' : 6,
    'MainOccupantRoomFloorArea': 30.0,
    'OtherOccupantRoomFloowArea': 60.0,
    'NonOccupantRoomFloorArea': 30.0,
}

data1_common

{'MainOccupantRoomFloorArea': 30.0,
 'NonOccupantRoomFloorArea': 30.0,
 'OtherOccupantRoomFloowArea': 60.0,
 'Region': 6}

In [4]:
def convert_1to2_common(d1):
    return {
        'Region'                     : d1['Region'],
        'FlagSimplifiedEnveopeInput' : True,
        'MainOccupantRoomFloorArea'  : d1['MainOccupantRoomFloorArea'],
        'OtherOccupantRoomFloowArea' : d1['OtherOccupantRoomFloowArea'],
        'NonOccupantRoomFloorArea'   : d1['NonOccupantRoomFloorArea']
           }

In [5]:
data2_common = convert_1to2_common(data1_common)
data2_common

{'FlagSimplifiedEnveopeInput': True,
 'MainOccupantRoomFloorArea': 30.0,
 'NonOccupantRoomFloorArea': 30.0,
 'OtherOccupantRoomFloowArea': 60.0,
 'Region': 6}

関数の実行

In [20]:
convert_1to2_common({
    'Region' : 6,
    'MainOccupantRoomFloorArea': 30.0,
    'OtherOccupantRoomFloowArea': 60.0,
    'NonOccupantRoomFloorArea': 30.0,
})

{'FlagSimplifiedEnveopeInput': True,
 'MainOccupantRoomFloorArea': 30.0,
 'NonOccupantRoomFloorArea': 30.0,
 'OtherOccupantRoomFloowArea': 60.0,
 'Region': 6}

#### コンバート【分類：壁体構成】

- レベル1：
  - 壁体名称$[-]$：$name$
  - 部位種別$[-]$：$type$
  - 平均熱貫流率$[W/(m^2・K)]$：$U_a$
- レベル2：
  - 壁体名称$[-]$：$name$
  - 部位種別$[-]$：$type$
  - <font color="red">部材名称$[-]$：$matrial$</font>
  - <font color="red">厚さ$[m]$：$d$</font>
  - <font color="red">熱伝達率$[W/(m・K)]$：$λ$</font>
  - <font color="red">容積比熱$[J/(m^3・K)]$：$Cρ$</font>
  - 平均熱貫流率$[W/(m^2・K)]$：$U_a$
  - <font color="red">一般部熱貫流率$[W/(m^2・K)]$：$U_g$</font>
  - <font color="red">一般部構造種別$[-]$：$Structure$</font>
  - <font color="red">木造熱橋部計算方法$[-]$：$HeatBridge$</font>
  - <font color="red">熱橋部熱面積比率$[-]$：$a_{hb}$</font>
  - <font color="red">熱橋部熱貫流率$[W/(m^2・K)]$：$U_{hb}$</font>
  - <font color="red">熱橋長さ$[m]$：$L_i$</font>
  - <font color="red">線熱貫流率$[W/(m・K)]$：$Ψ_i$</font>
  - <font color="red">補正熱貫流率$[W/(m^2・K)]$：$U_r$</font>

In [1]:
data1_wall = {
    'name': 'Sample Wall',
    'type': 2,
    'Ua': 0.538
}

data1_wall

{'Ua': 0.538, 'name': 'Sample Wall', 'type': 2}

In [12]:
def convert_1to2_wall(d1):
    import nbimporter    
    #import U_Value_to_Detail_Specification
    #Layers=U_Value_to_Detail_Specification.convert_1to2_wall(d1)
    from U_Value_to_Detail_Specification import convert_1to2_wall
    Layers=convert_1to2_wall(d1)
   
    return {'name': d1['name'],
            'type': d1['type'],
            'Ua': d1['Ua'],
            'Layers':Layers}

In [22]:
data2_wall=convert_1to2_wall(data1_wall)
data2_wall

{'Layers': {'Layers': [{'Cρ': 720, 'd': 0.012, 'material': 'PED', 'λ': 0.16},
   {'Cρ': 13, 'd': 0.064, 'material': 'GW16K', 'λ': 0.045},
   {'Cρ': 0, 'd': 1, 'material': 'Air', 'λ': 11.11},
   {'Cρ': 830, 'd': 0.0095, 'material': 'GPB', 'λ': 0.22}],
  'Ua': 0.538,
  'name': 'Sample Wall',
  'type': 2},
 'Ua': 0.538,
 'name': 'Sample Wall',
 'type': 2}

#### コンバート【分類：開口部】

- レベル1：
  - 開口部名称$[-]$：$name$
  - 日射熱取得率（夏期）$[-]$：$eta_c$
  - 日射熱取得率（冬期）$[-]$：$eta_h$
  - 熱貫流率$[W/(m^2・K)]$：$U$ 
- レベル2：
  - 開口部名称$[-]$：$name$
  - <font color="red">日射熱取得率$[-]$：$eta$</font>
  - 日射熱取得率（夏期）$[-]$：$eta_c$
  - 日射熱取得率（冬期）$[-]$：$eta_h$
  - 熱貫流率$[W/(m^2・K)]$：$U$ 

In [36]:
data1_window = {
    'name': 'Sample Window',
    'etac': 0.738,
    'etah': 0.738,
    'U': 4.65
}

data1_window

{'U': 4.65, 'etac': 0.738, 'etah': 0.738, 'name': 'Sample Window'}

In [16]:
def convert_1to2_window(d1):
    return {'name': d1['name'],
            'etac': d1['etac'],
            'etah': d1['etah'],
            'U': d1['U']}

In [38]:
data2_window=convert_1to2_window(data1_window)
data2_window

{'U': 4.65, 'etac': 0.738, 'etah': 0.738, 'name': 'Sample Window'}

#### コンバート【分類：外部日除け】

- レベル1：
  - 外部日除け名称$[-]$：$name$
  - 取得日射熱補正係数（夏期）$[-]$：$f_c$
  - 取得日射熱補正係数（冬期）$[-]$：$f_h$
- レベル2：
  - 外部日除け名称$[-]$：$name$
  - <font color="red">開口部高さ$[m]$：$y_2$</font>
  - <font color="red">出巾$[m]$：$z$</font>
  - <font color="red">窓上端と外部日除けの距離$[m]$：$y_1$</font>
  - 取得日射熱補正係数（夏期）$[-]$：$f_c$
  - 取得日射熱補正係数（冬期）$[-]$：$f_h$

In [18]:
data1_sunshade = {
    'name': 'Sample Sunshade',
    'fc': 0.700,
    'fh': 0.600,
}

data1_sunshade

{'fc': 0.7, 'fh': 0.6, 'name': 'Sample Sunshade'}

In [19]:
def convert_1to2_sunshade(d1):
    return {'name': d1['name'],
            'fc': d1['fc'],
            'fh': d1['fh']}

In [29]:
data2_sunshade=convert_1to2_sunshade(data1_sunshade)
data2_sunshade

{'fc': 0.7, 'fh': 0.6, 'name': 'Sample Sunshade'}

#### コンバート【分類：部位情報】

- レベル1：
  - 外皮フラグ$[-]$：$FlagPerimeter$
  - 日射フラグ$[-]$：$FlagSolarRadiation$
  - 方位$[-]$：$direction$
  - 非定常フラグ$[-]$：$FlagUnsteady$
  - 壁体・開口部名称$[-]$：$name$
  - 部位面積$[m^2]$：$A$
  - 外部日除け名称$[-]$：$sunshade$
- レベル2：
  - 外皮フラグ$[-]$：$FlagPerimeter$
  - 日射フラグ$[-]$：$FlagSolarRadiation$
  - 方位$[-]$：$direction$
  - 非定常フラグ$[-]$：$FlagUnsteady$
  - 壁体・開口部名称$[-]$：$name$
  - 部位面積$[m^2]$：$A$
  - 外部日除け名称$[-]$：$sunshade$

In [32]:
data1_element = {
    'name': 'Sample Element',
    'FlagPerimeter': 1,
    'FlagSolarRadiation': 1,
    'FlagUnsteady': 1,
    'direction': 1,
    'A': 1
}

data1_element

{'A': 1,
 'FlagPerimeter': 1,
 'FlagSolarRadiation': 1,
 'FlagUnsteady': 1,
 'direction': 1,
 'name': 'Sample Element'}

In [35]:
def convert_1to2_element(d1,d1_common):
    FloorArea=sum([d1_common['MainOccupantRoomFloorArea'],
                   d1_common['OtherOccupantRoomFloowArea'],
                   d1_common['NonOccupantRoomFloorArea']])
    return {'name': d1['name'],
            'FlagPerimeter': d1['FlagPerimeter'],
            'FlagSolarRadiation': d1['FlagSolarRadiation'],
            'FlagUnsteady': d1['FlagUnsteady'],
            'direction': d1['direction'],
            'A': round(d1['A']*FloorArea/90,2)}

In [32]:
data2_element=convert_1to2_element(data1_element,data1_common)
data2_element

{'A': 1.33,
 'FlagPerimeter': 1,
 'FlagSolarRadiation': 1,
 'FlagUnsteady': 1,
 'direction': 1,
 'name': 'Sample Element'}